## Market Cap for 2017-2019 for 200 coins as well as total market Cap

In [22]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
from time import sleep

# Create date range for historical snapshots
Date = pd.date_range(start='20170101', end='20191120', freq='7D').strftime('%Y%m%d')

result = np.empty((201,1))
for date in Date:

    # Retrieve historical snapshot data from date
    page = requests.get('https://coinmarketcap.com/historical/'+date)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # Extract html containers for each coin
    coin_containers = soup.find_all('tr', class_ = 'cmc-table-row')
    
    # Extract total Market Cap of the day
    totalMarketCapStr = soup.find('div', class_ = 'cmc-table-listing-footer sc-1jhjsq2-0 iInShT').text
    totalMarketCapStr = totalMarketCapStr[totalMarketCapStr.index('$'):]
    totalMarketCap = float(re.sub(r',|\$', '', totalMarketCapStr.strip()))
    
    daily = (date,totalMarketCap)
    for coin in coin_containers:

        coin_name = coin.find('td', class_ = 'cmc-table__cell cmc-table__cell--sortable cmc-table__cell--left cmc-table__cell--sort-by__symbol').text
        
        mktCapStr = coin.find('td', class_ = 'cmc-table__cell cmc-table__cell--sortable cmc-table__cell--right cmc-table__cell--sort-by__volume-24-h').text
        
        # convert the string into float
        mktCap = float(re.sub(r',|\$', '', mktCapStr.strip()))    

        daily = np.vstack((daily, (coin_name, mktCap)))
    
    
    result = np.hstack((result, daily))
    
    sleep(20)
# Delete the first redundant column
result = result[:,1:]
    
# Create data frame of data
df = pd.DataFrame(result)

# Write data to file
df.to_csv('total_market_cap17-19.csv', index = False)    
    